In [1]:
import findspark
findspark.init()

In [2]:
# import Spark Session from pyspark.sql
from pyspark.sql import SparkSession

In [3]:
# create my_spark sparkSession 
# SparkSession.builder.getOrCreate()
my_spark = SparkSession.builder.getOrCreate()

In [4]:
# print my_spark 
print(my_spark)

### Viewing tables
SparkSession은 catalog를 가지고 있다 \
catalog 란?\
=> 클러스터 내부 모든 데이터 리스트\
=> .listTables() => 모든 테이블 이름 리스트 출력 \
=> spark.catalog.listTables()

In [7]:
# print the tables in the catalog 
print(my_spark.catalog.listTables())

[]


In [8]:
spark = SparkSession.builder()\
    .appName("")

In [9]:
# load data 
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [10]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [11]:
# Create table 
user_log.createOrReplaceTempView("user_log_table")

In [12]:
# apply query
query = '''
    SELECT *
    FROM user_log_table
    LIMIT 10;
'''

In [17]:
user_log_10 = spark.sql(query)

In [18]:
user_log_10.show(10)

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

### Pandafy a spark dataframe

    => spark.toPandas()

In [23]:
query = '''
    SELECT artist, COUNT(*) as artist_count
    FROM user_log_table 
    GROUP BY artist
    ORDER BY artist_count desc;
'''

In [24]:
# run the query 
artist_counts = spark.sql(query)

In [25]:
# convert the results to a pandas dataframe 
artist_counts_pd = artist_counts.toPandas()

In [26]:
# print the head 
artist_counts_pd.head()

,artist,artist_count
0,None,1653
1,Coldplay,83
2,Kings Of Leon,69
3,Florence + The Machine,52
4,BjÃÂ¶rk,46


### Put some Spark in your data

In [29]:
import pandas as pd
import numpy as np


In [30]:
# create random pd_temp 
pd_temp = pd.DataFrame(np.random.random(10))

In [31]:
pd_temp

,0
0,0.820515
1,0.975538
2,0.752919
3,0.768485
4,0.253651
5,0.115129
6,0.108374
7,0.754835
8,0.831849
9,0.687697


In [32]:
# create spark_temp from pd_temp 
# spark.createDataFrame(pd_temp)
spark_temp = spark.createDataFrame(pd_temp)

In [34]:
spark_temp.show()

+-------------------+
|                  0|
+-------------------+
| 0.8205150631920037|
|  0.975538043384088|
| 0.7529192819001671|
| 0.7684850592578836|
| 0.2536512519605756|
|0.11512944719954976|
|0.10837405093038888|
| 0.7548351246169582|
| 0.8318492889791586|
| 0.6876968869449681|
+-------------------+



In [35]:
spark.catalog.listTables()

[Table(name='user_log_table', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [36]:
# add spark_temp to the catalog 
spark_temp.createOrReplaceTempView("temp")

In [37]:
# checking catalog 
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='user_log_table', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### Creating Columns
    df = df.withColumn("newcol", df.oldCol+1)

In [38]:
# create the dataframe temp 
df_temp = spark.table("temp")

In [41]:
df_temp.show()

+-------------------+
|                  0|
+-------------------+
| 0.8205150631920037|
|  0.975538043384088|
| 0.7529192819001671|
| 0.7684850592578836|
| 0.2536512519605756|
|0.11512944719954976|
|0.10837405093038888|
| 0.7548351246169582|
| 0.8318492889791586|
| 0.6876968869449681|
+-------------------+



In [44]:
# add new column 
df_temp = df_temp.withColumn("new col1",df_temp["0"]+1)

In [45]:
df_temp.show()

+-------------------+------------------+
|                  0|          new col1|
+-------------------+------------------+
| 0.8205150631920037|1.8205150631920037|
|  0.975538043384088|1.9755380433840881|
| 0.7529192819001671| 1.752919281900167|
| 0.7684850592578836|1.7684850592578836|
| 0.2536512519605756|1.2536512519605756|
|0.11512944719954976|1.1151294471995499|
|0.10837405093038888|1.1083740509303888|
| 0.7548351246169582|1.7548351246169582|
| 0.8318492889791586|1.8318492889791584|
| 0.6876968869449681| 1.687696886944968|
+-------------------+------------------+



## filtering

In [46]:
user_log.show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [47]:
df = user_log.toPandas()

In [48]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Showaddywaddy,Logged In,Kenneth,M,112,Matthews,232.93342,paid,"Charlotte-Concord-Gastonia, NC-SC",PUT,NextSong,1.509380e+12,5132,Christmas Tears Will Fall,200,1513720872284,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",1046
1,Lily Allen,Logged In,Elizabeth,F,7,Chase,195.23873,free,"Shreveport-Bossier City, LA",PUT,NextSong,1.512719e+12,5027,Cheryl Tweedy,200,1513720878284,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",1000
2,Cobra Starship Featuring Leighton Meester,Logged In,Vera,F,6,Blackwell,196.20526,paid,"Racine, WI",PUT,NextSong,1.499856e+12,5516,Good Girls Go Bad (Feat.Leighton Meester) (Alb...,200,1513720881284,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2219
3,Alex Smoke,Logged In,Sophee,F,8,Barker,405.99465,paid,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",PUT,NextSong,1.513010e+12,2372,Don't See The Point,200,1513720905284,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",2373
4,None,Logged In,Jordyn,F,0,Jones,NaN,free,"Syracuse, NY",GET,Home,1.513649e+12,1746,None,200,1513720913284,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",1747


In [49]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



### Filtering data 
    .filter("length > 200")\
    .filter(user_log.length > 200)


In [50]:
# filter user_log length by passing a string 
length_1 = user_log.filter("length > 200")

In [51]:
# filter by passing column 
length_2 = user_log.filter(user_log.length > 200)

In [52]:
length_1.show()

+----------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|   Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|      Alex Smoke|Logged In|   Sophee|     F|            8|   Barker|405.99465| paid|San Luis Obispo-P...|   PUT|NextSong|1513009647

In [53]:
length_2.show()

+----------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|   Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|      Alex Smoke|Logged In|   Sophee|     F|            8|   Barker|405.99465| paid|San Luis Obispo-P...|   PUT|NextSong|1513009647

### Selecting Data
    .select()
    .withColumn()

In [54]:
# artist, song, userId
# select the columns 

selected1 = user_log.select("artist","song","userId")

In [55]:
selected1.show()

+--------------------+--------------------+------+
|              artist|                song|userId|
+--------------------+--------------------+------+
|       Showaddywaddy|Christmas Tears W...|  1046|
|          Lily Allen|       Cheryl Tweedy|  1000|
|Cobra Starship Fe...|Good Girls Go Bad...|  2219|
|          Alex Smoke| Don't See The Point|  2373|
|                null|                null|  1747|
|                null|                null|  1747|
|              Redman|        Smoke Buddah|  1162|
|     Ulrich Schnauss|           On My Own|  1061|
|                null|                null|   748|
|                null|                null|   597|
|               Jay-Z|Heart Of The City...|  1806|
|         Evanescence|    Bring Me To Life|   748|
|     Scissor Sisters|               Laura|  1176|
|        3 Doors Down|    Here Without You|  2164|
|       George Younce|This Old House w/...|  2146|
|              Aly-Us|Follow Me (Club Mix)|  2219|
|                null|         

In [75]:
# select , df.Colname 
temp = user_log.select(user_log.artist, user_log.song, user_log.userId)

In [76]:
temp.show()

+--------------------+--------------------+------+
|              artist|                song|userId|
+--------------------+--------------------+------+
|       Showaddywaddy|Christmas Tears W...|  1046|
|          Lily Allen|       Cheryl Tweedy|  1000|
|Cobra Starship Fe...|Good Girls Go Bad...|  2219|
|          Alex Smoke| Don't See The Point|  2373|
|                null|                null|  1747|
|                null|                null|  1747|
|              Redman|        Smoke Buddah|  1162|
|     Ulrich Schnauss|           On My Own|  1061|
|                null|                null|   748|
|                null|                null|   597|
|               Jay-Z|Heart Of The City...|  1806|
|         Evanescence|    Bring Me To Life|   748|
|     Scissor Sisters|               Laura|  1176|
|        3 Doors Down|    Here Without You|  2164|
|       George Younce|This Old House w/...|  2146|
|              Aly-Us|Follow Me (Club Mix)|  2219|
|                null|         

In [72]:
# filtering userID== 1046
filter1 = user_log.userId == "1046"

In [71]:
filter2 = user_log.artist == "Showaddywaddy"

In [77]:
# filter the data 
selected3 = temp.filter(filter1).filter(filter2)

In [78]:
selected3.show()

+-------------+--------------------+------+
|       artist|                song|userId|
+-------------+--------------------+------+
|Showaddywaddy|Christmas Tears W...|  1046|
+-------------+--------------------+------+



### Selecting 2 
    df.select(df.colName.alias())
    df.selectExpr(df.colName + 1 as newcolname)

In [80]:
temp = user_log.select(user_log.artist, user_log.song, user_log.length, user_log.userId)

In [81]:
temp.show()

+--------------------+--------------------+---------+------+
|              artist|                song|   length|userId|
+--------------------+--------------------+---------+------+
|       Showaddywaddy|Christmas Tears W...|232.93342|  1046|
|          Lily Allen|       Cheryl Tweedy|195.23873|  1000|
|Cobra Starship Fe...|Good Girls Go Bad...|196.20526|  2219|
|          Alex Smoke| Don't See The Point|405.99465|  2373|
|                null|                null|     null|  1747|
|                null|                null|     null|  1747|
|              Redman|        Smoke Buddah|154.53995|  1162|
|     Ulrich Schnauss|           On My Own|402.93832|  1061|
|                null|                null|     null|   748|
|                null|                null|     null|   597|
|               Jay-Z|Heart Of The City...|223.26812|  1806|
|         Evanescence|    Bring Me To Life|237.11302|   748|
|     Scissor Sisters|               Laura|218.01751|  1176|
|        3 Doors Down|  

In [82]:
# define length_min 
length_min = (temp.length/60).alias("min")

In [84]:
# select with new column min 
temp = user_log.select("artist","song","length","userId",length_min)

In [85]:
temp.show()

+--------------------+--------------------+---------+------+------------------+
|              artist|                song|   length|userId|               min|
+--------------------+--------------------+---------+------+------------------+
|       Showaddywaddy|Christmas Tears W...|232.93342|  1046| 3.882223666666667|
|          Lily Allen|       Cheryl Tweedy|195.23873|  1000|3.2539788333333335|
|Cobra Starship Fe...|Good Girls Go Bad...|196.20526|  2219| 3.270087666666667|
|          Alex Smoke| Don't See The Point|405.99465|  2373|6.7665774999999995|
|                null|                null|     null|  1747|              null|
|                null|                null|     null|  1747|              null|
|              Redman|        Smoke Buddah|154.53995|  1162|2.5756658333333333|
|     Ulrich Schnauss|           On My Own|402.93832|  1061| 6.715638666666666|
|                null|                null|     null|   748|              null|
|                null|                nu

In [86]:
# create the same table using a SQL expression 
temp2 = user_log.selectExpr("artist","song","length","length/60 as min")

In [87]:
temp2.show()

+--------------------+--------------------+---------+------------------+
|              artist|                song|   length|               min|
+--------------------+--------------------+---------+------------------+
|       Showaddywaddy|Christmas Tears W...|232.93342| 3.882223666666667|
|          Lily Allen|       Cheryl Tweedy|195.23873|3.2539788333333335|
|Cobra Starship Fe...|Good Girls Go Bad...|196.20526| 3.270087666666667|
|          Alex Smoke| Don't See The Point|405.99465|6.7665774999999995|
|                null|                null|     null|              null|
|                null|                null|     null|              null|
|              Redman|        Smoke Buddah|154.53995|2.5756658333333333|
|     Ulrich Schnauss|           On My Own|402.93832| 6.715638666666666|
|                null|                null|     null|              null|
|                null|                null|     null|              null|
|               Jay-Z|Heart Of The City...|223.2681

### Aggregating
    .min(), max(), .count()

In [99]:
temp = user_log.filter(user_log.length.isNotNull()).select(user_log.artist, user_log.song, user_log.length, user_log.userId)

In [100]:
temp.show()

+--------------------+--------------------+---------+------+
|              artist|                song|   length|userId|
+--------------------+--------------------+---------+------+
|       Showaddywaddy|Christmas Tears W...|232.93342|  1046|
|          Lily Allen|       Cheryl Tweedy|195.23873|  1000|
|Cobra Starship Fe...|Good Girls Go Bad...|196.20526|  2219|
|          Alex Smoke| Don't See The Point|405.99465|  2373|
|              Redman|        Smoke Buddah|154.53995|  1162|
|     Ulrich Schnauss|           On My Own|402.93832|  1061|
|               Jay-Z|Heart Of The City...|223.26812|  1806|
|         Evanescence|    Bring Me To Life|237.11302|   748|
|     Scissor Sisters|               Laura|218.01751|  1176|
|        3 Doors Down|    Here Without You|237.13914|  2164|
|       George Younce|This Old House w/...|191.68608|  2146|
|              Aly-Us|Follow Me (Club Mix)|383.58159|  2219|
|            BjÃÂ¶rk|                Undo|348.57751|  2904|
|      David Bromberg|Sh

In [107]:
filter_1 = user_log.length.isNotNull()

In [118]:
user_log.filter(filter_1).show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [119]:
temp = user_log.filter(filter_1)

In [120]:
temp.show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [124]:
filter2 = temp.userId == 1046

In [135]:
user_log.filter(filter_1).filter(user_log.userId == "1046").groupBy().min("length").show()

+-----------+
|min(length)|
+-----------+
|  153.59955|
+-----------+



In [132]:
user_log.filter("length is not null").filter("userId == 1046").groupBy().min("length").show()

+-----------+
|min(length)|
+-----------+
|  153.59955|
+-----------+



TypeError: 'Column' object is not callable

In [137]:
user_log.filter(user_log.artist == "Showaddywaddy").show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [141]:
# avg song length of userId == 1046 

filter_1 = user_log.length.isNotNull()
filter_2 = user_log.userId == "1046"

In [143]:
user_log.filter(filter_1).filter(filter_2).groupBy().avg("length").show()

+------------------+
|       avg(length)|
+------------------+
|237.65578222222223|
+------------------+



### Grouping and Aggregating 1
    => .groupBy()

In [144]:
# group by userId
by_user_id = user_log.groupBy("userId")

In [147]:
by_user_id.show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [146]:
by_user_id.count().show()

+------+-----+
|userId|count|
+------+-----+
|  2904|    1|
|   691|    4|
|  2294|   62|
|  2162|   39|
|  1436|    2|
|  2088|   14|
|  2275|    5|
|  2756|    8|
|   800|    3|
|  1394|   28|
|   926|    7|
|  1746|    2|
|  2696|    2|
|  1280|    2|
|   870|    2|
|     7|    2|
|  1903|    7|
|   591|    1|
|   613|    4|
|   574|    7|
+------+-----+
only showing top 20 rows



In [148]:
# avg count, dd
by_user_id.avg("length").show()

+------+------------------+
|userId|       avg(length)|
+------+------------------+
|  2904|         348.57751|
|   691|269.66158666666666|
|  2294| 253.2148934545454|
|  2162|243.81803382352942|
|  1436|        316.695055|
|  2088|254.61907692307693|
|  2275|390.73043333333334|
|  2756|215.32689600000003|
|   800|         258.58567|
|  1394|230.37041076923074|
|   926|271.96036000000004|
|  1746|              null|
|  2696|         200.95955|
|  1280|              null|
|   870|        231.757915|
|     7|266.97097499999995|
|  1903|211.76379000000003|
|   591|         219.79383|
|   613|        209.632195|
|   574|       321.6387275|
+------+------------------+
only showing top 20 rows

